In [27]:

import os
os.getcwd() 
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import warnings
from test import *
warnings.filterwarnings("ignore")
from datasets import load_dataset,load_metric
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch_optimizer as optim
from transformers.optimization import Adafactor, AdafactorSchedule
import torch.backends.cudnn as cudnn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler, SubsetRandomSampler
from torch.autograd import Variable
import logging
import sys
import transformers
from basic_model import *
import time
import argparse
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
import string
from cycle import *
from train import *
from utils import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('t5-small')
def d(l):
    return tokenizer.batch_decode(l,skip_special_tokens=True)
def en(l):
    return tokenize(l,tokenizer,512,True)

In [33]:
label = [[ 1363,  1661,     6, 14595,     6,    27,   133,   114,    12,  2763,
          8667,   180, 10363,    15,    26,   449,    21,    46,  1287,   934,
             5,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [  451,    65,  2767,   139,     8,   962,    16,   128,  4963,    11,
            16,     8,  4492,  5054,   255,   808,   905,    13,   186,    13,
             8, 12123,     7,    24,    43,   118,   953,    26,  1918,    48,
           934,     5,     1,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0]]
pred = [[ 1363,  1661,     6, 10989,    11,  7569,   904,     6,    27,   133,
           114,    12,  2763,  8667,   180, 10363,    15,    26,   449,    21,
             8,  1345,   934,     5,     1,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [   94,    19,  1176,    24,    34,    19,  1176,    24,    34,    19,
          1176,    24,    34,    19,  1176,    24,    34,    19,  1176,    24,
            34,    19,  1176,    24,    34,    19,  1176,    24,    34,    19,
          1176,    24,    34,    19,  1176,    24,    34,    19,  1176,    24,
            34,    19,  1176,    24,    34,    19,  1176,    24,    34,    19,
          1176,    24,    34,    19,  1176,    24,    34,    19,  1176,    24,
            34,    19,  1176,    24,    34,    19,  1176,    24,    34,    19,
          1176,    24,    34,    19,  1176,    24,    34,    19,  1176,    24,
            34,    19,  1176,    24,    34,    19,  1176,    24,    34,    19,
          1176,    24,    34,    19,  1176,    24,    34,    19,  1176,    24,
            34,    19,  1176,    24,    34,    19,  1176,    24,    34,    19,
          1176,    24,    34,    65,  1026,     3,     9,   381,    13, 12123,
             7,    24,    43,   118,     3, 23505,     5,     1]]
pred = torch.Tensor(pred).long().cuda()

['Mr President, ladies and gentlemen, I would like to thank Mrs Schroedter for the sound report.',
 'It is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it is true that it has taken a number of amendments that have been drafted.']

In [28]:
model = torch.load('./model/D_B.pt').train()

In [29]:

label = en(['I have some sympathy with the last speaker: sometimes the Committee on Budgetary Control gets so involved with the important work they are doing, that they are unable to see the wood for the trees.', 'I also welcome the presentation on behalf of the Legal Affairs Committee and the Economic and Monetary Affairs Committee which took a balanced approach to the major and indeed legitimate concerns of the Committee on Budgetary Control about fraud.'])[0]
# [           'I agree with the previous speaker: sometimes the Committee on Budgetary Control is so deep in its important work that it does not see the forest behind trees.', 'I also welcome the statements made on behalf of the Committee on Legal Affairs and the Internal Market and the Committee on Economic and Monetary Affairs, which took a fairly balanced position on the serious and quite justified concerns of the Committee on Budgetary Control with regard to fraud.']

pred = en([ 'I agree with the previous speaker: sometimes the Committee on Budgetary Control is so deep in its important work that it does not see the forest behind trees.', 'I also welcome the statements made on behalf of the Committee on Legal Affairs and the Internal Market and the Committee on Economic and Monetary Affairs, which have taken a fairly balanced position on the serious and quite justified concerns of the Committee on Budgetary Control with regard to fraud.'])[0]

In [30]:
#real sentences
x = label.cuda()
#delte 0 give huge diff


# x = torch.Tensor(x).long().cuda()
print(x)
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_label =  model.classifier(distr)#(bs,1)
print(ret_label)

print(model(x,x_attn))

tensor([[   27,    43,   128, 27907,    28,     8,   336,  5873,    10,  1664,
             8,  3201,    30, 12532,  1208,  4330,  2347,    78,  1381,    28,
             8,   359,   161,    79,    33,   692,     6,    24,    79,    33,
             3,  6319,    12,   217,     8,  1679,    21,     8,  3124,     5,
             1,     0,     0,     0,     0],
        [   27,    92,  2222,     8,  3831,    30,  6089,    13,     8, 11281,
         12078,  3201,    11,     8,  9071,    11,  1290,  1582,  1208, 12078,
          3201,    84,   808,     3,     9,  8965,  1295,    12,     8,   779,
            11,  5071, 12372,  3315,    13,     8,  3201,    30, 12532,  1208,
          4330,    81,  7712,     5,     1]], device='cuda:0')
torch.Size([2, 45])
		---------embedding
torch.Size([2, 45, 512])
tensor([[[ 15.8255,   3.0661,  14.9297,  ..., -11.0776,  -4.4553,   5.7272],
         [ 16.4459, -13.6218, -23.6599,  ...,   8.4843,   0.2263,  -0.3465],
         [  1.5718,   1.1976,  16.3511, 

In [31]:
#real sentences
x = pred.cuda()
       
# x = torch.Tensor(x).long().cuda()
print(x.shape)
x_attn = (x>0.5).long()

print('\t\t---------embedding')
x_emb = model.embedding(x)
print(x_emb.shape)
print(x_emb)# same for padding

print('\t\t---------encode')
distr = model.encoder(inputs_embeds=x_emb,attention_mask=x_attn).last_hidden_state#(bs,sentence length,512)
print(distr.shape)
print(distr)

print('\t\t---------mask the vector')
x_attn_unsq= x_attn.unsqueeze(-1)
distr = torch.mul(distr,x_attn_unsq)#previously
print(distr.shape)
print(distr)


print('\t\t---------mean of the unmask ')
print(torch.sum(distr,1).shape)
print(torch.sum(x_attn,1).shape)
distr = torch.sum(distr,1)/torch.sum(x_attn_unsq,1)
print(distr.shape)
print(distr)

print('\t\t---------classifer ')
ret_pred =  model.classifier(distr)#(bs,1)
print(ret_pred)


print(model(x,x_attn))

torch.Size([2, 57])
		---------embedding
torch.Size([2, 57, 512])
tensor([[[ 15.8255,   3.0661,  14.9297,  ..., -11.0776,  -4.4553,   5.7272],
         [  2.9129, -16.4970, -32.1548,  ..., -16.8586, -30.2383,  18.4215],
         [ -7.3098,   4.8835,   7.0894,  ...,  19.4460,  -4.3243,  17.3978],
         ...,
         [ -1.9713,   0.1810,  -7.0931,  ...,  -0.4844,   2.7078,  -2.8701],
         [ -1.9713,   0.1810,  -7.0931,  ...,  -0.4844,   2.7078,  -2.8701],
         [ -1.9713,   0.1810,  -7.0931,  ...,  -0.4844,   2.7078,  -2.8701]],

        [[ 15.8255,   3.0661,  14.9297,  ..., -11.0776,  -4.4553,   5.7272],
         [ 12.8770,   4.6584,  -1.4246,  ...,  -0.2715, -33.7260,  34.3136],
         [ 24.1588,  -6.9753, -34.5270,  ..., -18.4298,  21.7420,   5.4933],
         ...,
         [-10.7290,  -9.2744, -27.7144,  ...,   5.9364,  -4.2273,   0.3175],
         [ -4.3332,   7.3672, -14.5854,  ...,  13.0432,  -2.7207,  -3.1233],
         [ 12.6463,   8.1982, -11.5313,  ...,   7.8180,  

In [32]:
criterionGAN = torch.nn.MSELoss()
pred_real = ret_label
print(pred_real)
loss_D_real = criterionGAN(pred_real, torch.ones((pred_real.shape[0],1),device='cuda'))
print(loss_D_real)
# Fake
pred_fake = ret_pred
loss_D_fake = criterionGAN(pred_fake, torch.zeros((pred_fake.shape[0],1),device='cuda'))
# Combined loss and calculate gradients
print(loss_D_fake)


loss_D = (loss_D_real + loss_D_fake) * 0.5
print(loss_D)

tensor([[0.1755],
        [0.1934]], device='cuda:0', grad_fn=<AddmmBackward0>)
tensor(0.6651, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.0317, device='cuda:0', grad_fn=<MseLossBackward0>)
tensor(0.3484, device='cuda:0', grad_fn=<MulBackward0>)


,attention_mask=x_attn is essential for the encoder inpuit


## We previously use torch.mean to (x after embedding)*x_attn but the mean will be small for the long sentences, now we use torch.sum(distr,1)/torch.sum(x_attn,1)

## We can see that the D judge the sentence by it's length
## small output for the first part, but large for the latter part, so the longer the higher

## so we change the output to [batch,0,512] ie, onlythe first word's sumQ*V and than droppout and then classifier

## FAKE dont end with 1, cuz i use model.generate but not self.generate